# ETL de datos de importación de productos

## Instalación de librerías base

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import uuid

## Extraction

In [2]:
import pandas as pd
import psycopg2 as pg

# Conexión a la base de datos PostgreSQL
conn = pg.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="mysecretpass"
)

# Consulta SQL para seleccionar los datos
query = "SELECT * FROM trades"

# Tamaño del lote para cargar los datos
chunk_size = 1000

# Crear un generador de pandas para leer los datos en lotes
trades_generator = pd.read_sql_query(query, conn, chunksize=chunk_size)

# Concatenar los DataFrames de los lotes en uno solo
trades_df = pd.concat(trades_generator)

# Mostrar las primeras 5 filas del DataFrame completo


In [3]:
df_countries = pd.read_json('src/country_data.json')

In [4]:
df_codes = pd.read_csv('src/hs-codes.csv')

In [9]:
df_parents = df_codes[df_codes['Level']==2].copy()
df_parents

,Order,Level,Code,Parent,Code_comm,Parent.1,Description_complex,Description
2,1654557,2,10021000090,1.001100e+10,1,I,CHAPTER 1 - LIVE ANIMALS,LIVE ANIMALS
52,1654607,2,20021000090,1.001100e+10,2,I,CHAPTER 2 - MEAT AND EDIBLE MEAT OFFAL,MEAT AND EDIBLE MEAT OFFAL
140,1654695,2,30021000090,1.001100e+10,3,I,"CHAPTER 3 - FISH AND CRUSTACEANS, MOLLUSCS AND...","FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT..."
416,1654971,2,40021000090,1.001100e+10,4,I,CHAPTER 4 - DAIRY PRODUCE; BIRDS' EGGS; NATURA...,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...
463,1655018,2,50021000090,1.001100e+10,5,I,"CHAPTER 5 - PRODUCTS OF ANIMAL ORIGIN, NOT ELS...","PRODUCTS OF ANIMAL ORIGIN, NOT ELSEWHERE SPECI..."
...,...,...,...,...,...,...,...,...
7238,1661793,2,930021000090,9.300110e+11,93,XIX,CHAPTER 93 - ARMS AND AMMUNITION; PARTS AND AC...,ARMS AND AMMUNITION; PARTS AND ACCESSORIES THE...
7264,1661819,2,940021000090,9.400110e+11,94,XX,"CHAPTER 94 - FURNITURE; BEDDING, MATTRESSES, M...","FURNITURE; BEDDING, MATTRESSES, MATTRESS SUPPO..."
7319,1661874,2,950021000090,9.400110e+11,95,XX,"CHAPTER 95 - TOYS, GAMES AND SPORTS REQUISITES...","TOYS, GAMES AND SPORTS REQUISITES; PARTS AND A..."
7362,1661917,2,960021000090,9.400110e+11,96,XX,CHAPTER 96 - MISCELLANEOUS MANUFACTURED ARTICLES,MISCELLANEOUS MANUFACTURED ARTICLES


## Transform

#### Clean codes

In [14]:
df_code = df_codes[df_codes['Code_comm'].notnull()]

In [15]:
def clean_code(text):
    text = str(text)
    parent_code = None

    if len(text) == 12:
        code = text[:5]
        parent_code = text[:1]
    else:
        code = text[:6]
        parent_code = text[:2]

    try:
        parent = df_parents[df_parents['Code_comm'] == parent_code]['Description'].values[0]
    except:
        parent = None


    return (code, parent)

In [17]:
df_codes[['clean_code', 'parent_description']] = df_codes.apply(
    lambda x:  clean_code( x['Code'] ),
    axis=1,
    result_type='expand'
)

In [ ]:
df_codes = df_codes[df_codes['clean_code'].notnull()][]

### Clean Countries

### Merge

### Clean trades

## Load

In [ ]:
import os
import boto3

